In [23]:
import requests
import pandas as pd
from dataclasses import dataclass
import warnings
import spacy
from typing import Protocol
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from typing import Type
from typing import Type
from functools import lru_cache


import unicodedata


warnings.filterwarnings("ignore")
pd.options.display.max_columns = 100


In [24]:
@dataclass
class MeLiClient:
    site: str

    def get_categories(self) -> pd.DataFrame:
        url = f"https://api.mercadolibre.com/sites/{self.site}/categories"
        return pd.DataFrame(requests.get(url).json())

    def get_items_in_category(self, category_id: str, offset=0) -> pd.DataFrame:

        url = f"https://api.mercadolibre.com/sites/{self.site}/search?category={category_id}&offset={offset}"
        request = requests.get(url)
        items = request.json()
        try:
            return pd.DataFrame(items["results"])
        except Exception:
            return pd.DataFrame()

    def get_all_items_in_category(self, category_id: str) -> pd.DataFrame:

        offset = 0
        results = []

        result = self.get_items_in_category(category_id)
        results.append(result)

        while len(result) > 0:
            offset += 50

            result = self.get_items_in_category(category_id, offset=offset)
            results.append(result)

        return pd.concat(results)


site = "MCO"
meli = MeLiClient(site)


In [25]:
categories = meli.get_categories()
categories.query("id=='MCO1000'")


,id,name
17,MCO1000,"Electrónica, Audio y Video"


In [26]:
# https://developers.mercadolibre.com.ar/es_co/atributos
# http://www.heikopaulheim.com/docs/swj2018_product_data.pdf

# items[items["attributes"].apply(lambda sku: 'DESCRIPTIVE_TAGS' in (x["id"] for x in sku))] # empty
# items[items["attributes"].apply(lambda sku: 'PRODUCT_FEATURES' in (x["id"] for x in sku))] # empty


In [27]:
def get_attribute_value(attributes: list, attribute_name: str):
    return next(
        (x["value_name"] for x in attributes if x["id"] == attribute_name),
        None,
    )


def build_dataset(
    meli: MeLiClient,
    category_id: str,
    features: list[str],
    attributes: list[str],
    save=True,
):

    items = meli.get_all_items_in_category(category_id)
    df = items[features]

    for attribute in attributes:
        df[attribute.lower()] = items.attributes.apply(
            get_attribute_value, attribute_name=attribute
        )

    df = df.drop(columns="attributes").dropna(how="all", axis=1).reset_index(drop=True)

    if save:
        df.to_csv(f"{category_id}_items.csv")

    return df


FEATURES = ["id", "title", "thumbnail", "domain_id", "attributes"]
INTERESTING_ATTRIBUTES = ["GTIN", "BRAND", "MODEL"]
category = "MCO1000"  # categories.id.sample().squeeze()
print(category)


df = build_dataset(meli, category, FEATURES, INTERESTING_ATTRIBUTES)
df


MCO1000


,id,title,thumbnail,domain_id,brand,model
0,MCO618049088,Audífonos Sony Zx Series Mdr-zx110 Negro,http://http2.mlstatic.com/D_975252-MLA45258514...,MCO-HEADPHONES,Sony,MDR-ZX110
1,MCO627599757,Mini Drone Dji Mavic Mini 2 Drdji018 Fly More ...,http://http2.mlstatic.com/D_888397-MLA47979312...,MCO-DRONES,DJI,Mini 2
2,MCO613351936,Audífonos In-ear Jbl Tune 110 Black,http://http2.mlstatic.com/D_798368-MLA46444369...,MCO-HEADPHONES,JBL,110
3,MCO838086582,Soporte North Bayou Nb-p4 De Pared Para Tv/mon...,http://http2.mlstatic.com/D_831097-MLA46523626...,MCO-TV_AND_MONITOR_MOUNTS,North Bayou,NB-P4
4,MCO838336512,Soporte North Bayou Nb F80 De Mesa Para Tv/mon...,http://http2.mlstatic.com/D_701870-MLA46522721...,MCO-TV_AND_MONITOR_MOUNTS,North Bayou,NB F80
...,...,...,...,...,...,...
1045,MCO548760665,Cable Hdmi 30 Metros Mallado Doble Filtro Punt...,http://http2.mlstatic.com/D_920327-MCO40429690...,MCO-AUDIO_AND_VIDEO_CABLES_AND_ADAPTERS,HDMI,30M
1046,MCO829996289,Roku Streaming Stick 4k Versión 2021 Smart Tv ...,http://http2.mlstatic.com/D_755105-MCO48162443...,MCO-STREAMING_MEDIA_DEVICES,Roku,3820R
1047,MCO499242229,Sensor Inductivo Sensor Proximidad Lj12a3-4-z/...,http://http2.mlstatic.com/D_782190-MCO43648332...,MCO-INDUCTIVE_SENSORS,Generico,EL DESCRITO EN LA PUBLICACION
1048,MCO545193597,Energeizer Pila Alkalina Grande Tipo D X 2 Und,http://http2.mlstatic.com/D_977856-MCO44257523...,MCO-CELL_BATTERIES,Energizer,ALKALINA


In [28]:
# Global Preprocessing Utils

nlp = spacy.load("es_core_news_md")


def preprocessor(text: str):
    text = unicodedata.normalize("NFKD", text)
    return text.lower().strip()


def tokenizer(text: str):
    doc = nlp(text)  # probably overkill using spacy for this but ok
    tokens = [word.lemma_ for word in doc if not word.is_stop and not word.is_punct]
    return tokens


In [29]:
class Embedding(Protocol):
    @classmethod
    def initialize(cls, corpus: list[str]) -> None:
        ...

    def similarity(self, other: "Embedding") -> float:
        ...


In [30]:
class SpacyEmbedding:
    def __init__(self, text: str):
        self.embedding = nlp(text)

    @classmethod
    def initialize(cls, corpus: list[str]) -> None:
        pass

    def similarity(self, other: "SpacyEmbedding") -> float:
        return self.embedding.similarity(other.embedding)

    def __repr__(self):
        return str(tuple(self.embedding))


In [31]:
class BagOfWordsEmbedding:
    def __init__(self, text: str):
        self.embedding = self.bow.transform([text])[0]
        self.tokens = tokenizer(text)

    @classmethod
    def initialize(cls, corpus: list[str]) -> None:
        cls.bow = CountVectorizer(tokenizer=tokenizer, preprocessor=preprocessor).fit(
            corpus
        )

    def similarity(self, other: "BagOfWordsEmbedding") -> float:
        return cosine_similarity(self.embedding, other.embedding)[0][0]

    def __repr__(self):
        return str(tuple(self.tokens))


In [32]:
class TFIDFEmbedding:
    def __init__(self, text: str):
        self.embedding = self.tfidf.transform([text])[0]
        self.tokens = tokenizer(text)

    @classmethod
    def initialize(cls, corpus: list[str]) -> None:
        cls.tfidf = TfidfVectorizer(tokenizer=tokenizer, preprocessor=preprocessor).fit(
            corpus
        )

    def similarity(self, other: "TFIDFEmbedding") -> float:
        return cosine_similarity(self.embedding, other.embedding)[0][0]

    def __repr__(self):
        return str(tuple(self.tokens))


In [33]:
def find_embeddings(df, embedding_type: Type[Embedding], embedding_cols: tuple[str]):

    embedding_type.initialize(df.title)
    for col in embedding_cols:
        df[f"{col}_embedding"] = df[col].apply(embedding_type)
    return df


EMBEDDING_COLS = ("title",)  # , "brand", "model"]
df = find_embeddings(df, BagOfWordsEmbedding, EMBEDDING_COLS)
df


,id,title,thumbnail,domain_id,brand,model,title_embedding
0,MCO618049088,Audífonos Sony Zx Series Mdr-zx110 Negro,http://http2.mlstatic.com/D_975252-MLA45258514...,MCO-HEADPHONES,Sony,MDR-ZX110,"('Audífonos', 'Sony', 'Zx', 'Series', 'Mdr-zx1..."
1,MCO627599757,Mini Drone Dji Mavic Mini 2 Drdji018 Fly More ...,http://http2.mlstatic.com/D_888397-MLA47979312...,MCO-DRONES,DJI,Mini 2,"('Mini', 'Drone', 'Dji', 'Mavic', 'Mini', '2',..."
2,MCO613351936,Audífonos In-ear Jbl Tune 110 Black,http://http2.mlstatic.com/D_798368-MLA46444369...,MCO-HEADPHONES,JBL,110,"('Audífonos', 'in-ear', 'Jbl', 'Tune', '110', ..."
3,MCO838086582,Soporte North Bayou Nb-p4 De Pared Para Tv/mon...,http://http2.mlstatic.com/D_831097-MLA46523626...,MCO-TV_AND_MONITOR_MOUNTS,North Bayou,NB-P4,"('Soporte', 'North', 'Bayou', 'Nb-p4', 'Pared'..."
4,MCO838336512,Soporte North Bayou Nb F80 De Mesa Para Tv/mon...,http://http2.mlstatic.com/D_701870-MLA46522721...,MCO-TV_AND_MONITOR_MOUNTS,North Bayou,NB F80,"('Soporte', 'North', 'Bayou', 'Nb', 'F80', 'Me..."
...,...,...,...,...,...,...,...
1045,MCO548760665,Cable Hdmi 30 Metros Mallado Doble Filtro Punt...,http://http2.mlstatic.com/D_920327-MCO40429690...,MCO-AUDIO_AND_VIDEO_CABLES_AND_ADAPTERS,HDMI,30M,"('Cable', 'Hdmi', '30', 'Metros', 'Mallado', '..."
1046,MCO829996289,Roku Streaming Stick 4k Versión 2021 Smart Tv ...,http://http2.mlstatic.com/D_755105-MCO48162443...,MCO-STREAMING_MEDIA_DEVICES,Roku,3820R,"('Roku', 'Streaming', 'Stick', '4k', 'Versión'..."
1047,MCO499242229,Sensor Inductivo Sensor Proximidad Lj12a3-4-z/...,http://http2.mlstatic.com/D_782190-MCO43648332...,MCO-INDUCTIVE_SENSORS,Generico,EL DESCRITO EN LA PUBLICACION,"('Sensor', 'Inductivo', 'Sensor', 'Proximidad'..."
1048,MCO545193597,Energeizer Pila Alkalina Grande Tipo D X 2 Und,http://http2.mlstatic.com/D_977856-MCO44257523...,MCO-CELL_BATTERIES,Energizer,ALKALINA,"('Energeizer', 'Pila', 'Alkalina', 'grande', '..."


In [34]:
def find_similarities(
    df: pd.DataFrame, embedding: Embedding, feature="title_embedding"
):
    similarities = df[feature].apply(embedding.similarity)
    return similarities


find_similarities(df, df.title_embedding.iloc[0])


0       1.000000
1       0.000000
2       0.166667
3       0.105409
4       0.102062
          ...   
1045    0.000000
1046    0.000000
1047    0.000000
1048    0.000000
1049    0.000000
Name: title_embedding, Length: 1050, dtype: float64

In [35]:
def feature_exact_match(df: pd.DataFrame, feature: str, feature_value):
    return df[feature] == feature_value


feature_exact_match(df, "brand", "Hill's")


0       False
1       False
2       False
3       False
4       False
        ...  
1045    False
1046    False
1047    False
1048    False
1049    False
Name: brand, Length: 1050, dtype: bool

In [36]:
def find_similar_products(
    df: pd.DataFrame,
    product_id: str,
    embedding_feature="title_embedding",
    penalty_features=["brand", "domain_id"],
    penalty_value=0.1,
    threshold=0.7,
):
    details = df.copy()
    product = df.set_index("id").loc[product_id]
    details["similarity"] = find_similarities(
        df, product[embedding_feature], embedding_feature
    )
    details["raw_similarity"] = details.similarity.copy()

    for penalty_feature in penalty_features:
        is_exact_match = feature_exact_match(
            df, penalty_feature, product[penalty_feature]
        )
        details[f"{penalty_feature}_penalty"] = (
            ~is_exact_match * penalty_value if pd.notna(product[penalty_feature]) else 0
        )
        details["similarity"] -= details[f"{penalty_feature}_penalty"]

    products_above_threshold = details[details["similarity"] > threshold]
    similar_products = details.loc[products_above_threshold.index]
    return similar_products, details


PENALTY_FEATURES = ("brand", "domain_id", "model")
find_similar_products(df, "MCO618049088", penalty_features=PENALTY_FEATURES)[0]


,id,title,thumbnail,domain_id,brand,model,title_embedding,similarity,raw_similarity,brand_penalty,domain_id_penalty,model_penalty
0,MCO618049088,Audífonos Sony Zx Series Mdr-zx110 Negro,http://http2.mlstatic.com/D_975252-MLA45258514...,MCO-HEADPHONES,Sony,MDR-ZX110,"('Audífonos', 'Sony', 'Zx', 'Series', 'Mdr-zx1...",1.000000,1.000000,0.0,0.0,0.0
14,MCO618050413,Audífonos Sony Zx Series Mdr-zx110 Blanco,http://http2.mlstatic.com/D_896816-MLA45258465...,MCO-HEADPHONES,Sony,MDR-ZX110,"('Audífonos', 'Sony', 'Zx', 'Series', 'Mdr-zx1...",0.833333,0.833333,0.0,0.0,0.0
19,MCO618050281,Audífonos Sony Zx Series Mdr-zx110 Rosa,http://http2.mlstatic.com/D_639965-MLA45258510...,MCO-HEADPHONES,Sony,MDR-ZX110,"('Audífonos', 'Sony', 'Zx', 'Series', 'Mdr-zx1...",0.833333,0.833333,0.0,0.0,0.0
31,MCO619299293,Audífonos Sony Zx Series Mdr-zx110ap Negro,http://http2.mlstatic.com/D_708548-MLA45258577...,MCO-HEADPHONES,Sony,MDR-ZX110AP,"('Audífonos', 'Sony', 'Zx', 'Series', 'Mdr-zx1...",0.733333,0.833333,0.0,0.0,0.1


In [37]:
def load_test_dataset(filename: str):
    test_data = pd.read_excel(filename, sheet_name=["test_set", "all"])
    test_set, all = test_data["test_set"], test_data["all"]
    return test_set, all


In [38]:
@dataclass(frozen=True)
class SimilarityConfig:
    test_filename: str
    embedding_type: Type[Embedding]
    embedding_feature: str
    penalty_features: tuple[str]
    penalty_value: float
    threshold: float


In [39]:
config = SimilarityConfig(
    test_filename="test_electronica.xlsx",
    embedding_type=BagOfWordsEmbedding,
    embedding_feature="title",
    penalty_features=PENALTY_FEATURES,
    penalty_value=0.05,
    threshold=0.7,
)


In [40]:
@lru_cache(16)
def build_testing_data(test_filename, embedding_type, embedding_feature):
    test_set, df = load_test_dataset(test_filename)
    data = find_embeddings(df, embedding_type, [embedding_feature])
    return test_set, data


test_set, data = build_testing_data(
    config.test_filename, config.embedding_type, config.embedding_feature
)


In [41]:
def test_product_matches(
    df: pd.DataFrame, matches: pd.DataFrame, true_matches: list[str]
):

    tp = matches[matches.id.isin(true_matches)]
    fp = matches[~matches.id.isin(true_matches)]
    fn = df[(df.id.isin(true_matches) & (~df.id.isin(matches.id)))]
    tn = df[(~df.id.isin(true_matches) & (~df.id.isin(matches.id)))]

    return tp, fp, fn, tn


test_id = test_set.test_id.iloc[0]
tm = test_set[test_set.test_id == test_id].id
matches, details = find_similar_products(
    df,
    test_id,
    penalty_features=config.penalty_features,
    embedding_feature=config.embedding_feature + "_embedding",
    penalty_value=config.penalty_value,
    threshold=config.threshold,
)
test_product_matches(data, matches, tm)


(              id                                      title  \
 0   MCO618049088   Audífonos Sony Zx Series Mdr-zx110 Negro   
 14  MCO618050413  Audífonos Sony Zx Series Mdr-zx110 Blanco   
 19  MCO618050281    Audífonos Sony Zx Series Mdr-zx110 Rosa   
 
                                             thumbnail       domain_id brand  \
 0   http://http2.mlstatic.com/D_975252-MLA45258514...  MCO-HEADPHONES  Sony   
 14  http://http2.mlstatic.com/D_896816-MLA45258465...  MCO-HEADPHONES  Sony   
 19  http://http2.mlstatic.com/D_639965-MLA45258510...  MCO-HEADPHONES  Sony   
 
         model                                    title_embedding  similarity  \
 0   MDR-ZX110  ('Audífonos', 'Sony', 'Zx', 'Series', 'Mdr-zx1...    1.000000   
 14  MDR-ZX110  ('Audífonos', 'Sony', 'Zx', 'Series', 'Mdr-zx1...    0.833333   
 19  MDR-ZX110  ('Audífonos', 'Sony', 'Zx', 'Series', 'Mdr-zx1...    0.833333   
 
     raw_similarity  brand_penalty  domain_id_penalty  model_penalty  
 0         1.000000    

In [44]:
def get_precision_and_recall(
    true_positives: pd.DataFrame,
    false_positives: pd.DataFrame,
    false_negatives: pd.DataFrame,
    true_negatives: pd.DataFrame,
):
    tp, fp, fn, tn = (
        len(true_positives),
        len(false_positives),
        len(false_negatives),
        len(true_negatives),
    )
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn)
    return precision, recall


get_precision_and_recall(*test_product_matches(data, matches, tm))


(0.75, 1.0)

In [49]:
def test_similarity_model(cfg: SimilarityConfig):
    print(f"building testing data for {cfg}...")
    test_set, data = build_testing_data(
        cfg.test_filename, cfg.embedding_type, cfg.embedding_feature
    )
    test_products = test_set.test_id.unique()
    all_cf = []
    all_matches = []

    print("finding similar products...")
    for product_id in test_products:
        print(f"\ntesting {product_id}")
        true_matches = test_set[test_set.test_id == product_id].id
        matches, details = find_similar_products(
            data,
            product_id,
            penalty_features=cfg.penalty_features,
            embedding_feature=cfg.embedding_feature + "_embedding",
            penalty_value=cfg.penalty_value,
            threshold=cfg.threshold,
        )
        confusion_matrix = test_product_matches(data, matches, true_matches)
        precision, recall = get_precision_and_recall(*confusion_matrix)
        print(f"{precision=}")
        print(f"{recall=}")

        all_cf.append(confusion_matrix)
        all_matches.append(matches)

    print("finished\n")

    global_confusion_matrix = [pd.concat(df) for df in zip(*all_cf)]
    global_precision, global_recall = get_precision_and_recall(*global_confusion_matrix)
    global_matches = pd.concat(all_matches)
    print(f"{global_precision=}")
    print(f"{global_recall=}")

    return global_matches, global_precision, global_recall


config = SimilarityConfig(
    test_filename="test_electronica.xlsx",
    embedding_type=BagOfWordsEmbedding,
    embedding_feature="title",
    penalty_features=PENALTY_FEATURES,
    penalty_value=0.15,
    threshold=0.7,
)

test_similarity_model(config)


building testing data for SimilarityConfig(test_filename='test_electronica.xlsx', embedding_type=<class '__main__.BagOfWordsEmbedding'>, embedding_feature='title', penalty_features=('brand', 'domain_id', 'model'), penalty_value=0.15, threshold=0.7)...
finding similar products...

testing MCO618049088
precision=1.0
recall=1.0

testing MCO641446794
precision=1.0
recall=1.0

testing MCO593426721
precision=1.0
recall=0.5

testing MCO626370015
precision=1.0
recall=0.6666666666666666

testing MCO453389071
precision=1.0
recall=0.6666666666666666
finished

global_precision=1.0
global_recall=0.6875


(               id                                              title  \
 0    MCO618049088           Audífonos Sony Zx Series Mdr-zx110 Negro   
 14   MCO618050413          Audífonos Sony Zx Series Mdr-zx110 Blanco   
 16   MCO618050281            Audífonos Sony Zx Series Mdr-zx110 Rosa   
 92   MCO641446794   Audífonos Audio-technica M-series Ath-m30x Negro   
 46   MCO593426721  Cinta Led Multicolor Rgb Luz 5mts + Control + ...   
 73   MCO607187019  Cinta Led Multicolor Rgb Luz 5mts + Control + ...   
 977  MCO856218109  Cinta Led Multicolor Rgb Luz 5mts + Control + ...   
 58   MCO626370015  Google Nest Mini 2nd Gen Con Asistente Virtual...   
 111  MCO853335104  Google Nest Mini 2nd Gen Con Asistente Virtual...   
 294  MCO453389071  Cable Hdmi 5 Metros Doble Filtro Mallado Punta...   
 516  MCO555797022  Cable Hdmi 5 Metros Doble Filtro Mallado Punta...   
 
                                              thumbnail  \
 0    http://http2.mlstatic.com/D_975252-MLA45258514...   
 14 

In [50]:
@dataclass
class ExperimentResults:
    experiment: SimilarityConfig
    precision: float
    recall: float


def run_experiments(experiments: list[SimilarityConfig]):
    results = []
    for experiment in experiments:
        _, precision, recall = test_similarity_model(experiment)
        results.append(ExperimentResults(experiment, precision, recall))

    return results


THRESHOLDS = [0.7, 0.8, 0.9]
EMBEDDING_TYPES = [BagOfWordsEmbedding, TFIDFEmbedding, SpacyEmbedding]

PENALTY_FEATURES = ("brand", "domain_id", "model")
TEST_FILENAME = "test_electronica.xlsx"
EMBEDDING_FEATURE = "title"
PENALTY_VALUE = 0.15

experiments = [
    SimilarityConfig(
        TEST_FILENAME, et, EMBEDDING_FEATURE, PENALTY_FEATURES, PENALTY_VALUE, th
    )
    for et in EMBEDDING_TYPES
    for th in THRESHOLDS
]
results = run_experiments(experiments)
results


building testing data for SimilarityConfig(test_filename='test_electronica.xlsx', embedding_type=<class '__main__.BagOfWordsEmbedding'>, embedding_feature='title', penalty_features=('brand', 'domain_id', 'model'), penalty_value=0.15, threshold=0.7)...
finding similar products...

testing MCO618049088
precision=1.0
recall=1.0

testing MCO641446794
precision=1.0
recall=1.0

testing MCO593426721
precision=1.0
recall=0.5

testing MCO626370015
precision=1.0
recall=0.6666666666666666

testing MCO453389071
precision=1.0
recall=0.6666666666666666
finished

global_precision=1.0
global_recall=0.6875
building testing data for SimilarityConfig(test_filename='test_electronica.xlsx', embedding_type=<class '__main__.BagOfWordsEmbedding'>, embedding_feature='title', penalty_features=('brand', 'domain_id', 'model'), penalty_value=0.15, threshold=0.8)...
finding similar products...

testing MCO618049088
precision=1.0
recall=1.0

testing MCO641446794
precision=1.0
recall=1.0

testing MCO593426721
precisi

[ExperimentResults(experiment=SimilarityConfig(test_filename='test_electronica.xlsx', embedding_type=<class '__main__.BagOfWordsEmbedding'>, embedding_feature='title', penalty_features=('brand', 'domain_id', 'model'), penalty_value=0.15, threshold=0.7), precision=1.0, recall=0.6875),
 ExperimentResults(experiment=SimilarityConfig(test_filename='test_electronica.xlsx', embedding_type=<class '__main__.BagOfWordsEmbedding'>, embedding_feature='title', penalty_features=('brand', 'domain_id', 'model'), penalty_value=0.15, threshold=0.8), precision=1.0, recall=0.5),
 ExperimentResults(experiment=SimilarityConfig(test_filename='test_electronica.xlsx', embedding_type=<class '__main__.BagOfWordsEmbedding'>, embedding_feature='title', penalty_features=('brand', 'domain_id', 'model'), penalty_value=0.15, threshold=0.9), precision=1.0, recall=0.375),
 ExperimentResults(experiment=SimilarityConfig(test_filename='test_electronica.xlsx', embedding_type=<class '__main__.TFIDFEmbedding'>, embedding_fea

In [51]:
best_config = SimilarityConfig(
    test_filename="test_electronica.xlsx",
    embedding_type=BagOfWordsEmbedding,
    embedding_feature="title",
    penalty_features=("brand", "domain_id", "model"),
    penalty_value=0.15,
    threshold=0.7,
)

matches, precision, recall = test_similarity_model(best_config)
matches


building testing data for SimilarityConfig(test_filename='test_electronica.xlsx', embedding_type=<class '__main__.BagOfWordsEmbedding'>, embedding_feature='title', penalty_features=('brand', 'domain_id', 'model'), penalty_value=0.15, threshold=0.7)...
finding similar products...

testing MCO618049088
precision=1.0
recall=1.0

testing MCO641446794
precision=1.0
recall=1.0

testing MCO593426721
precision=1.0
recall=0.5

testing MCO626370015
precision=1.0
recall=0.6666666666666666

testing MCO453389071
precision=1.0
recall=0.6666666666666666
finished

global_precision=1.0
global_recall=0.6875


,id,title,thumbnail,domain_id,brand,model,title_embedding,similarity,raw_similarity,brand_penalty,domain_id_penalty,model_penalty
0,MCO618049088,Audífonos Sony Zx Series Mdr-zx110 Negro,http://http2.mlstatic.com/D_975252-MLA45258514...,MCO-HEADPHONES,Sony,MDR-ZX110,"('Audífonos', 'Sony', 'Zx', 'Series', 'Mdr-zx1...",1.000000,1.000000,0.00,0.0,0.00
14,MCO618050413,Audífonos Sony Zx Series Mdr-zx110 Blanco,http://http2.mlstatic.com/D_896816-MLA45258465...,MCO-HEADPHONES,Sony,MDR-ZX110,"('Audífonos', 'Sony', 'Zx', 'Series', 'Mdr-zx1...",0.833333,0.833333,0.00,0.0,0.00
16,MCO618050281,Audífonos Sony Zx Series Mdr-zx110 Rosa,http://http2.mlstatic.com/D_639965-MLA45258510...,MCO-HEADPHONES,Sony,MDR-ZX110,"('Audífonos', 'Sony', 'Zx', 'Series', 'Mdr-zx1...",0.833333,0.833333,0.00,0.0,0.00
92,MCO641446794,Audífonos Audio-technica M-series Ath-m30x Negro,http://http2.mlstatic.com/D_669027-MLA45258905...,MCO-HEADPHONES,Audio-Technica,ATH-M30x,"('Audífonos', 'Audio-technica', 'M-series', 'A...",1.000000,1.000000,0.00,0.0,0.00
46,MCO593426721,Cinta Led Multicolor Rgb Luz 5mts + Control + ...,http://http2.mlstatic.com/D_892089-MCO44006453...,MCO-LED_CHIPS,RGB 5050,RGB 5050,"('Cinta', 'Led', 'Multicolor', 'Rgb', 'Luz', '...",1.000000,1.000000,0.00,0.0,0.00
73,MCO607187019,Cinta Led Multicolor Rgb Luz 5mts + Control + ...,http://http2.mlstatic.com/D_691827-MCO45442068...,MCO-LED_CHIPS,Osblack,Led,"('Cinta', 'Led', 'Multicolor', 'Rgb', 'Luz', '...",0.700000,1.000000,0.15,0.0,0.15
977,MCO856218109,Cinta Led Multicolor Rgb Luz 5mts + Control + ...,http://http2.mlstatic.com/D_700055-MCO49035786...,MCO-LED_CHIPS,TecnoNovedadesCO,CLRGB060TN,"('Cinta', 'Led', 'Multicolor', 'Rgb', 'Luz', '...",0.700000,1.000000,0.15,0.0,0.15
58,MCO626370015,Google Nest Mini 2nd Gen Con Asistente Virtual...,http://http2.mlstatic.com/D_738549-MLA45993657...,MCO-SMART_SPEAKERS,Google,Nest Mini 2nd Gen,"('Google', 'Nest', 'Mini', '2nd', 'Gen', 'Asis...",1.000000,1.000000,0.00,0.0,0.00
111,MCO853335104,Google Nest Mini 2nd Gen Con Asistente Virtual...,http://http2.mlstatic.com/D_820774-MLA45993657...,MCO-SMART_SPEAKERS,Google,Nest Mini 2nd Gen,"('Google', 'Nest', 'Mini', '2nd', 'Gen', 'Asis...",0.923077,0.923077,0.00,0.0,0.00
294,MCO453389071,Cable Hdmi 5 Metros Doble Filtro Mallado Punta...,http://http2.mlstatic.com/D_885979-MCO48754809...,MCO-AUDIO_AND_VIDEO_CABLES_AND_ADAPTERS,hdmi,5 metros,"('Cable', 'Hdmi', '5', 'Metros', 'Doble', 'Fil...",1.000000,1.000000,0.00,0.0,0.00
